In [45]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandas as pd

In [119]:
data=pd.read_excel('titanic.xls')
data

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [120]:
data.drop(['body','name'],1,inplace=True)
data.apply(pd.to_numeric,errors='ignore')    #data.convert_objects(conver_numeric=True)
data.fillna(0,inplace=True)
data

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,female,14.5000,1,0,2665,14.4542,0,C,0,0
1305,3,0,female,0.0000,1,0,2665,14.4542,0,C,0,0
1306,3,0,male,26.5000,0,0,2656,7.2250,0,C,0,0
1307,3,0,male,27.0000,0,0,2670,7.2250,0,C,0,0


In [121]:
def handle_non_numerical_data(data):
    columns=data.columns.values
    print(columns)
    for column in columns:
        text_digit_vals={} #{'Female':0,'Male':1}
        def convert_to_int(val):
            return text_digit_vals[val]
        if data[column].dtype!=np.int64 and data[column].dtype !=np.float64:
            column_contents=data[column].values.tolist()
            unique_elements=set(column_contents)#unique values
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique]=x
                    x+=1
            
            data[column]=list(map(convert_to_int, data[column]))#for columns
    return data

In [122]:
new_data=handle_non_numerical_data(data)
new_data

['pclass' 'survived' 'sex' 'age' 'sibsp' 'parch' 'ticket' 'fare' 'cabin'
 'embarked' 'boat' 'home.dest']


,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,754,211.3375,156,3,1,273
1,1,1,0,0.9167,1,2,539,151.5500,148,3,26,185
2,1,0,1,2.0000,1,2,539,151.5500,148,3,0,185
3,1,0,0,30.0000,1,2,539,151.5500,148,3,0,185
4,1,0,1,25.0000,1,2,539,151.5500,148,3,0,185
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,1,14.5000,1,0,568,14.4542,0,1,0,0
1305,3,0,1,0.0000,1,0,568,14.4542,0,1,0,0
1306,3,0,0,26.5000,0,0,734,7.2250,0,1,0,0
1307,3,0,0,27.0000,0,0,34,7.2250,0,1,0,0


In [123]:
x=np.array(data.drop(['survived'],1).astype(float))
y=np.array(data['survived'])

In [124]:
classifier=KMeans(n_clusters=2)
classifier.fit(x)
correct=0
for i in range (len(x)):
    predict_me=np.array(x[i].astype(float))
    predict_me=predict_me.reshape(-1,len(predict_me))
    prediction=classifier.predict(predict_me)
    if prediction[0]==y[i]:
        correct+=1
print(correct/(len(x)))

0.5164247517188694


In [143]:
x=np.array(data.drop(['survived'],1).astype(float))
x=preprocessing.scale(x)#difference between preprocessing.scale()
y=np.array(data['survived'])

In [144]:
classifier=KMeans(n_clusters=2)
classifier.fit(x)
correct=0
for i in range (len(x)):
    predict_me=np.array(x[i].astype(float))
    predict_me=predict_me.reshape(-1,len(predict_me))
    prediction=classifier.predict(predict_me)
    if prediction[0]==y[i]:
        correct+=1
print(correct/(len(x)))

0.7372039724980901


In [139]:
data.drop(['boat'],1)
x=np.array(data.drop(['survived'],1).astype(float))
x=preprocessing.scale(x)
y=np.array(data['survived'])

In [140]:
classifier=KMeans(n_clusters=2)
classifier.fit(x)
correct=0
for i in range (len(x)):
    predict_me=np.array(x[i].astype(float))
    predict_me=predict_me.reshape(-1,len(predict_me))
    prediction=classifier.predict(predict_me)
    if prediction[0]==y[i]:
        correct+=1
print(correct/(len(x)))

0.7372039724980901


In [141]:
data.drop(['sex'],1)
x=np.array(data.drop(['survived'],1).astype(float))
x=preprocessing.scale(x)
y=np.array(data['survived'])

In [142]:
classifier=KMeans(n_clusters=2)
classifier.fit(x)
correct=0
for i in range (len(x)):
    predict_me=np.array(x[i].astype(float))
    predict_me=predict_me.reshape(-1,len(predict_me))
    prediction=classifier.predict(predict_me)
    if prediction[0]==y[i]:
        correct+=1
print(correct/(len(x)))

0.26279602750190983
